# Pandas I/O: trabalhando com diferentes formatos de arquivos

### Aula 1 - Fazendo leitura de arquivos csv

1. Verifique se o arquivo CSV está separado por vírgula ou ponto e vírgula.

2. A codificação do arquivo é ISO-8859-1.

3. As três primeiras linhas linhas do arquivo podem ser desconsideradas, pois o cabeçalho só começa na quarta linha.

4. As 9 últimas linhas também podem ser desconsideradas, pois são apenas informações sobre onde os dados foram obtidos.

5. Para deletar as últimas linhas é necessário adicionar o parâmetro engine='python'.

In [ ]:
import pandas as pd

try:
    data_sus = pd.read_csv('dados_sus.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3, skipfooter=9)
    print("Arquivo carregado com separador por vírgula.")
except:
    # Se falhar, tentando carregar o arquivo com separador por ponto e vírgula
    try:
        data_sus = pd.read_csv('dados_sus.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3, skipfooter=9)
        print("Arquivo carregado com separador por ponto e vírgula.")
    except Exception as e:
        print(f"Falha ao carregar o arquivo: {e}")

### Aula 2 - Utilizando planilhas

Chegou a hora de você testar os conhecimentos desenvolvidos durante a aula. Nós temos um link do Google Planilhas que contém dados importantes sobre as emissões de gás carbônico pelo mundo. O conjunto de dados foi obtido no Kaggle e consiste em emissões de CO2 per capita de todos os países do mundo de 1990 a 2019.

In [ ]:
# id da url da planilha
idplanilha = "1pvBoLyX8kP0TjtUbadVMGdTl4yzm6bHMThhPiqCVtpw"

# Endereço url google docs
url = f'https://docs.google.com/spreadsheets/d/{idplanilha}/gviz/tq?tqx=out:csv$sheet'

# Lendo o arquivo
dados = pd.read_csv(url)

# Verificando 
dados.head()

### Aula 3 - Manipulando arquivos JSON

Chegou a hora de testar os conhecimentos desenvolvidos durante a aula. O DataFrame mostrado abaixo foi gerado após a obtenção de dados da API Fruitvice:
Este DataFrame possui 6 colunas: genus (gênero), name (nome), id, family (família), order (ordem) e nutritions (nutrições). Note que a coluna nutritions está com todas as informações nutricionais aninhadas. Portanto, os dados precisam ser normalizados. O desafio agora é normalizar esse DataFrame.

In [ ]:
import requests
import json

# Request usando requests
df_frutas = requests.get('https://fruityvice.com/api/fruit/all')

# Lendo o arquivo json
df_frutas_json = json.loads(df_frutas.text)

# Normalizando o arquivo json
df_frutas_normalizado = pd.json_normalize(df_frutas_json)

# Verificando o arquivo json
df_frutas_normalizado.head()

### Aula 4 - Lendo dados em HTML e XML

Chegou a hora de testar os conhecimentos desenvolvidos durante a aula. Vanessa é uma cientista de dados que está realizando algumas análises com dados ambientais. Ela está desenvolvendo um projeto para avaliar o impacto ambiental das atividades humanas em diferentes países do mundo, mas para isso precisa das estimativas populacionais desses países. Ao pesquisar na internet, encontrou uma tabela de estimativas populacionais em um artigo da página Wikipédia. Assim como Vanessa, seu desafio é obter um DataFrame da tabela que contém as informações do número de habitantes de cada país.

In [ ]:
# Url 
url = "https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_popula%C3%A7%C3%A3o"

# Lendo a url
df = pd.read_html(url)[0]

# Revendo a coluna Unnamed
df = df.drop(columns=['Unnamed: 0']).reset_index(drop=True)

# Verificando o df
df.head()

### Aula 5 - Trabalhando com banco de dados

Chegou a hora de testar os conhecimentos desenvolvidos durante a aula. Você é responsável por criar um banco de dados local de clientes para uma instituição financeira. Temos o arquivo CSV com os dados de clientes. Sua missão é:

- Criar o banco de dados local com a biblioteca SQLAlchemy.
- Escrever os dados do arquivo CSV neste banco de dados local.
- Realizar três atualizações no banco de dados:
- Atualizar o registro do cliente de ID 6840104 que teve o rendimento anual alterado para 300000.
- Excluir o registro do cliente de ID 5008809, pois essa pessoa não possui mais conta na instituição financeira.
- Criar um novo registro de cliente seguindo as especificações abaixo:
- ID_Cliente: 6850985
- Idade: 33
- Grau_escolaridade: Doutorado
- Estado_civil: Solteiro
- Tamanho_familia: 1
- Categoria_de_renda: Empregado
- Ocupacao: TI
- Anos_empregado: 2
- Rendimento_anual: 290000
- Tem_carro: 0
- Moradia: Casa/apartamento próprio

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Boolean

# Ler o arquivo CSV
url = "https://raw.githubusercontent.com/alura-cursos/Pandas/main/clientes_banco.csv"
df = pd.read_csv(url)

# Criar o banco de dados local
engine = create_engine('sqlite:///clientes_banco.db')
df.to_sql('clientes', con=engine, if_exists='replace', index=False)

# Verificar os dados inseridos
df = pd.read_sql('clientes', con=engine)

df.head()

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import update, delete

# Configurar a sessão
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

# Definir o mapeamento da tabela
class Cliente(Base):
    __tablename__ = 'clientes'
    ID_Cliente = Column(Integer, primary_key=True)
    Idade = Column(Integer)
    Grau_escolaridade = Column(String)
    Estado_civil = Column(String)
    Tamanho_familia = Column(Integer)
    Categoria_de_renda = Column(String)
    Ocupacao = Column(String)
    Anos_empregado = Column(Float)
    Rendimento_anual = Column(Float)
    Tem_carro = Column(Boolean)
    Moradia = Column(String)

In [ ]:
# Atualizar o registro do cliente de ID 6840104
session.query(Cliente).filter(Cliente.ID_Cliente == 6840104).update({"Rendimento_anual": 300000})
session.commit()

In [ ]:
# Excluir o registro do cliente de ID 5008809
session.query(Cliente).filter(Cliente.ID_Cliente == 5008809).delete()
session.commit()

In [ ]:
# Criar um novo registro de cliente
novo_cliente = Cliente(
    ID_Cliente=6850985,
    Idade=33,
    Grau_escolaridade='Doutorado',
    Estado_civil='Solteiro',
    Tamanho_familia=1,
    Categoria_de_renda='Empregado',
    Ocupacao='TI',
    Anos_empregado=2,
    Rendimento_anual=290000,
    Tem_carro=0,
    Moradia='Casa/apartamento próprio'
)
session.add(novo_cliente)
session.commit()

In [ ]:
df = pd.read_sql('clientes', con=engine)

df.tail()